# Banned Books Gensim Word2Vec

**Summary:**
This notebook uses Gensim's Word2Vec to analyze banned books, exploring word embeddings, semantic similarity, and vocabulary patterns in the context of censorship and literary studies.

# Gensim Word2Vec

## Load Packages and Corpus

In [ ]:
import pickle

In [ ]:
!pip install gensim==4.3.2

In [ ]:
#import libraries

from nltk.tokenize import sent_tokenize
from nltk.tokenize.treebank import TreebankWordTokenizer
import nltk
import glob
from pathlib import Path
import os
import gensim
import pandas as pd
from gensim.models import Word2Vec
from tqdm import tqdm
import multiprocessing
import re
import string

In [ ]:
#extra
from gensim.test.utils import datapath
from gensim.test.utils import common_texts
from gensim.models import KeyedVectors

# Upload Data

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Subset_model_real_new.bin to Subset_model_real_new (1).bin
User uploaded file "Subset_model_real_new (1).bin" with length 1972877 bytes


# Load Model - NOT WORKING

In [ ]:
Subset_model = Word2Vec.load("/content/Subset_model_real_new_(1).bin")


FileNotFoundError: ignored

# Explore Model


## Get Similarities + Analogies

In [ ]:
Subset_model.wv.most_similar("river", topn=5)

In [ ]:
Subset_model.wv.similarity('water','toxic')

In [ ]:
result = Subset_model.wv.most_similar(positive=['river', 'canal'], negative=['ocean'])
print("{}: {:.4f}".format(*result[0]))

In [ ]:
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Setup for plot

base_words = ['toxic', 'pollution', 'river', 'water']
toxic_similar_S = [i[0] for i in Subset_model.wv.most_similar(positive='toxic', topn=5)]
pollution_similar_S = [i[0] for i in Subset_model.wv.most_similar(positive='pollution', topn=5)]
river_similar_S = [i[0] for i in Subset_model.wv.most_similar(positive='river', topn=5)]
water_similar_S = [i[0] for i in Subset_model.wv.most_similar(positive='water', topn=5)]


In [ ]:
all_words_S = np.hstack((base_words, toxic_similar_S, pollution_similar_S, river_similar_S, water_similar_S))
print(all_words_S)

In [ ]:
# Calculate similarities
base_words = ['toxic', 'pollution', 'river', 'water', 'sewer', 'sewage', 'cleanse', 'clean', 'dirty', 'foam', 'infect', 'ocean', 'sea']

similarities = []
for word in base_words:
    sim_words = Subset_model.wv.most_similar(word, topn=15)
    similarities.append(sim_words)

similar_w_df = pd.DataFrame(similarities)
similar_w_df = similar_w_df.T
similar_w_df.columns = base_words
similar_w_df.to_csv('subset_each_word_similarities.csv')

In [ ]:
# Calculate similarities
base_words = ['toxic', 'pollution', 'river', 'water', 'sewer', 'sewage', 'cleanse', 'clean', 'dirty', 'foam', 'infect', 'ocean', 'sea']

similarities = []
for word in base_words:
    for other_word in base_words:
        similarity_score = Subset_model.wv.similarity(word, other_word)
        similarities.append((word, other_word, similarity_score))

# Create a DataFrame
df = pd.DataFrame(similarities, columns=['Word 1', 'Word 2', 'Similarity'])
df

In [ ]:
df.to_csv('INSERT NAME.csv')

## Plot Similar Words

In [ ]:
#plot with color coding

labels = [i for i in all_words_S]
tokens = Subset_model.wv[labels]
tokens = tokens.astype(float)

tsne_model = TSNE(init='pca', learning_rate='auto', perplexity=15)
new_values = tsne_model.fit_transform(tokens)


x= []
y= []
for value in new_values:
    x.append(value[0])
    y.append(value[1])

for word in all_words_S:
    i=labels.index(word)
    plt.annotate(labels[i],
                xy=(x[i], y[i]),
                xytext=(5, 2),
                textcoords='offset points',
                ha='right',
                va='bottom')
    if word in toxic_similar_S:
        plt.scatter(x[i], y[i], color='gold')
    elif word == 'toxic':
         plt.scatter(x[i], y[i], color='goldenrod')
    elif word in pollution_similar_S:
         plt.scatter(x[i], y[i], color='indianred')
    elif word == 'pollution':
         plt.scatter(x[i], y[i], color='brown')
    elif word in river_similar_S:
         plt.scatter(x[i], y[i], color='silver')
    elif word == 'river':
         plt.scatter(x[i], y[i], color='grey')
    elif word in water_similar_S:
         plt.scatter(x[i], y[i], color='lightblue')
    elif word == 'water':
        plt.scatter(x[i], y[i], color='royalblue')
    plt.title("Word2Vec Embeddings from Water Novel Subset", fontweight='bold' )